In [145]:
data = pd.read_csv(r'E:\ML_study\学习笔记\推荐\movielens\ml-20m\ratings_small.csv', engine='python')
data = data.sample(frac=1)

In [150]:
train = data[:3500]
test = data[3500:]

In [154]:
train.to_csv(r'E:\ML_study\学习笔记\推荐\movielens\ml-20m\ratings_train.csv', encoding='utf-8', index=False)
test.to_csv(r'E:\ML_study\学习笔记\推荐\movielens\ml-20m\ratings_test.csv', encoding='utf-8', index=False)

In [1]:
from collections import defaultdict
import time
import gc
import pandas as pd
import numpy as np

from tqdm import tqdm

s = time.perf_counter()
train = pd.read_csv(r'E:\ML_study\学习笔记\推荐\movielens\ml-20m\ratings_train.csv', engine='python')
test = pd.read_csv(r'E:\ML_study\学习笔记\推荐\movielens\ml-20m\ratings_test.csv', engine='python')

def combine(data):
    item_users = dict()
    user_items = dict()
    users = set()
    data['userId'] = 'user_' + data['userId'].astype(str)
    data['movieId'] = 'movie_' + data['movieId'].astype(str)
    for u, item, r in data[['userId', 'movieId', 'rating']].values:
        item_users.setdefault(item, []).append((u, r))
        user_items.setdefault(u, []).append((item, r))
        users.add(u)
    del data
    gc.collect()
    return item_users, user_items, users

train_item_users, train_user_items, train_users = combine(train)
test_item_users, test_user_items, test_users = combine(test)
print('cost time %s min' % ((time.perf_counter() - s) / 60))

cost time 0.0017224516666666666 min


In [98]:
# from joblib import dump, load
# dump(item_users, r'D:\Users\hao.guo\比赛代码提炼\推荐系统\movielen\data\item_users.pkl')
# dump(users, r'D:\Users\hao.guo\比赛代码提炼\推荐系统\movielen\data\users.pkl')

In [2]:
import math
from tqdm import tqdm_notebook
def UserSimilarity(item_users, users):
    # build inverse table for item_users
#     item_users = dict()
#     users = set()
#     for u, items in data.items():
#         for i in items.keys():
#             # if i not in item_users:
#             #     item_users[i] = set()
#             item_users.setdefault(i, set()).add(u)
#         users.add(u)
    #calculate co-rated items between users
    # C = dict()
    C = defaultdict(dict)
    # C = np.zeros((len(item_users), len(item_users)))
    # user items list
    N = defaultdict(int)
    print('共现矩阵计算')
    for i, pairs in tqdm(item_users.items()):
        for u, r1 in pairs:
            N[u] += r1 ** 2
            for v, r2 in pairs:
                if u == v:
                    continue
                if v not in list(C[u]):
                    C[u][v] = 0
                C[u][v] += r1 * r2
    #calculate finial similarity matrix W
    W = defaultdict(dict)
    print('相似矩阵计算')
    for u, related_users in tqdm(C.items()):
        for v, cuv in related_users.items():
            W[u][v] = cuv / math.sqrt(N[u] * N[v])
    return W
s = time.perf_counter()
W = UserSimilarity(train_item_users, train_users)
print('cost time %s min' % ((time.perf_counter() - s) / 60))

共现矩阵计算


100%|███████████████████████████████████████████████████████████████████████████| 1694/1694 [00:00<00:00, 49970.12it/s]


相似矩阵计算


100%|███████████████████████████████████████████████████████████████████████████████| 50/50 [00:00<00:00, 16682.46it/s]


cost time 0.0009028316666666628 min


In [10]:
def Recommend(user, train, W, K):
    rank = defaultdict(int)
    interacted_items = list(map(lambda p: p[0], train[user]))
    for v, wuv in sorted(W[user].items(), key=lambda p: p[1], reverse=True)[0:K]:
        for i, rvi in train[v]:
            if i in interacted_items:
            #we should filter items user interacted before
                continue
        rank[i] += wuv * rvi
    return sorted(rank.items(), key=lambda p: p[0], reverse=True)[:50]

def Precision(train, test, W, N):
    hit = 0
    all = 0
    for user in train.keys():
        tu = list(map(lambda p: p[0], test[user]))
        rank = Recommend(user, train, W, N)
        for item, pui in rank:
            if item in tu:
                hit += 1
        all += 50
    return hit / (all * 1.0)

def Recall(train, test, W, N):
    hit = 0
    all = 0
    for user in train.keys():
        tu = list(map(lambda p: p[0], test[user]))
        rank = Recommend(user, train, W, N)
        for item, pui in rank:
            if item in tu:
                hit += 1
        all += len(tu)
    return hit / (all * 1.0)

def Coverage(train, test, W, N):
    recommend_items = set()
    all_items = set()
    for user in train.keys():
        for item, r in train[user]:
            all_items.add(item)
        rank = Recommend(user, train, W, N)
        for item, pui in rank:
            recommend_items.add(item)
    return len(recommend_items) / (len(all_items) * 1.0)

def Popularity(train, test, W, N):
    item_popularity = dict()
    for user, items in train.items():
        for item, r in items:
            if item not in item_popularity:
                item_popularity[item] = 0
            item_popularity[item] += 1
    ret = 0
    n = 0
    for user in train.keys():
        rank = Recommend(user, train, W, N)
        for item, pui in rank:
            ret += np.log(1 + item_popularity[item])
            n += 1
    ret /= n * 1.0
    return ret 

def eval(train_user_items, test_user_items, W, N):
    p = Precision(train_user_items, test_user_items, W, N)
    r = Recall(train_user_items, test_user_items, W, N)
    c = Coverage(train_user_items, test_user_items, W, N)
    pop = Popularity(train_user_items, test_user_items, W, N)
    return (p, r, c, pop)
    
print(eval(train_user_items, test_user_items, W, 5))
print(eval(train_user_items, test_user_items, W, 10))
print(eval(train_user_items, test_user_items, W, 20))
print(eval(train_user_items, test_user_items, W, 30))
print(eval(train_user_items, test_user_items, W, 40))
print(eval(train_user_items, test_user_items, W, 50))

(0.0048, 0.008005336891260841, 0.02715466351829988, 1.6236272299588879)
(0.0128, 0.02134756504336224, 0.02833530106257379, 1.624718973771883)
(0.0196, 0.03268845897264843, 0.028925619834710745, 1.6274730932508432)
(0.0248, 0.04136090727151434, 0.028925619834710745, 1.6056153223105636)
(0.0288, 0.04803202134756504, 0.028925619834710745, 1.5735711135252997)
(0.0304, 0.050700466977985324, 0.028925619834710745, 1.556790649912706)


In [164]:
Recommend('user_1', train_user_items, W, 5)

defaultdict(int,
            {'movie_1411': 0.5791108618080074,
             'movie_2093': 0.9568347360147121,
             'movie_3147': 0.7390869705076952,
             'movie_34': 0.5389405115000754,
             'movie_292': 0.41401278942708564})

In [167]:
train_user_items['user_1']

[('movie_2253', 3.5),
 ('movie_2683', 3.5),
 ('movie_2542', 4.0),
 ('movie_2644', 3.5),
 ('movie_1750', 3.5),
 ('movie_3499', 4.0),
 ('movie_2174', 4.0),
 ('movie_5952', 5.0),
 ('movie_6755', 3.5),
 ('movie_4128', 4.0),
 ('movie_6754', 4.0),
 ('movie_2193', 4.0),
 ('movie_2', 3.5),
 ('movie_2173', 4.0),
 ('movie_1089', 3.5),
 ('movie_7482', 3.0),
 ('movie_260', 4.0),
 ('movie_2762', 4.0),
 ('movie_2761', 3.0),
 ('movie_4911', 4.0),
 ('movie_7045', 3.5),
 ('movie_4226', 3.5),
 ('movie_6333', 4.0),
 ('movie_1243', 3.0),
 ('movie_1259', 4.0),
 ('movie_2118', 4.0),
 ('movie_2804', 3.5),
 ('movie_32', 3.5),
 ('movie_7454', 4.0),
 ('movie_5999', 3.5),
 ('movie_5146', 3.5),
 ('movie_4878', 3.5),
 ('movie_1350', 3.5),
 ('movie_1370', 3.0),
 ('movie_1200', 4.0),
 ('movie_4896', 4.0),
 ('movie_3153', 4.0),
 ('movie_3081', 4.0),
 ('movie_2194', 3.5),
 ('movie_367', 3.5),
 ('movie_3996', 4.0),
 ('movie_8961', 4.0),
 ('movie_253', 4.0),
 ('movie_3479', 4.0),
 ('movie_2628', 4.0),
 ('movie_3438', 3.